<a href="https://colab.research.google.com/github/honzas83/bio-bert/blob/main/bert_sample_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example usage of BERT in protein classification

## Required Python modules

In [1]:
%%capture pip_install

!pip install tensorflow-text \
             transformers==3.3.0 \
             pyyaml \
             sentencepiece \
             keras-bert==0.81.0 \
             keras-transformer==0.33.0 \
             pandas \
             scikit-learn==0.23 \
             biopython

     |████████████████████████████████| 3.4MB 8.7MB/s 
     |████████████████████████████████| 1.1MB 49.4MB/s 
     |████████████████████████████████| 1.2MB 55.8MB/s 
     |████████████████████████████████| 7.3MB 46.0MB/s 
     |████████████████████████████████| 2.3MB 57.1MB/s 
     |████████████████████████████████| 890kB 53.3MB/s 
     |████████████████████████████████| 3.0MB 45.6MB/s 
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=e4d89a61d9fa6b122d301e10ac953237e24b388592cf3bfaa499b9e65c852f7b
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.33.0-cp36-none-any.whl size=13261 sha256=2c948b3fff6e4df49d830fca62440e64791e6bc958c5ff44ef194d37a9d4c2d2
  Stored in directory: /root/.cache/pip/wheels/26/98/13/a28402939e1d48edd8704e6b02f223795af4a706815f4bf6d8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp

## Model download


In [2]:
!gdown --id 17FLvsbpjqR_SHAYY-S0YwAmjyLW2PYwx && unzip -u bert-pfam-10k.zip
!gdown --id 10jnY335GcVon8EGqo5h5Kxh_gjQoxbJH && unzip -u SCOPe.zip


Downloading...
From: https://drive.google.com/uc?id=17FLvsbpjqR_SHAYY-S0YwAmjyLW2PYwx
To: /content/bert-pfam-10k.zip
1.04GB [00:10, 98.6MB/s]
Archive:  bert-pfam-10k.zip
   creating: bert-pfam-10k/
  inflating: bert-pfam-10k/checkpoint  
  inflating: bert-pfam-10k/pfam.vocab  
  inflating: bert-pfam-10k/model.ckpt-1500000.index  
  inflating: bert-pfam-10k/model.ckpt-1500000.data-00000-of-00001  
  inflating: bert-pfam-10k/bert_config.json  
  inflating: bert-pfam-10k/pfam.model  
  inflating: bert-pfam-10k/model.ckpt-1500000.meta  
Downloading...
From: https://drive.google.com/uc?id=10jnY335GcVon8EGqo5h5Kxh_gjQoxbJH
To: /content/SCOPe.zip
14.2MB [00:00, 45.2MB/s]
Archive:  SCOPe.zip
   creating: SCOPe/
  inflating: SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.06.test.fa  
  inflating: SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.07.fa  
  inflating: SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.06.fa  
  inflating: SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.06.train.train

### Imports

In [3]:
import keras_bert
from keras_bert import load_trained_model_from_checkpoint

import sentencepiece 
import os
import pandas as pd
import numpy as np
import itertools
import random
from Bio import SeqIO

import tensorflow as tf
import keras
import keras.backend as K
from keras import layers
from keras.layers import Input, Dense, TimeDistributed, Bidirectional, LSTM, Concatenate, Conv1D, Dropout, Dot, Lambda, GlobalAvgPool1D, Add, GaussianNoise, Flatten
from keras.models import Model, load_model
from keras_bert.layers import Extract, MaskedGlobalMaxPool1D
from keras.regularizers import l1
from keras_bert import gelu
from keras_position_wise_feed_forward import FeedForward
from tensorflow.python.framework import tensor_shape
from keras.callbacks import LearningRateScheduler, EarlyStopping

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, average_precision_score

### Experiment configuration

In [4]:
# BERT pretrained model
CHECKPOINT_PATH = './bert-pfam-10k/'
CHECKPOINT_NAME = 'model.ckpt-1500000'
BERT_CONFIG = 'bert_config.json'

# Tokenization model for SentencePieces
SPM_MODEL = "./bert-pfam-10k/pfam.model"

# Load 12 layers from BERT
LAYER_NUM = 12

# Fine-tune the self-attention layers in FINE_TUNE
#FINE_TUNE = [10, 11, 12]
FINE_TUNE = []

# Use True to increase number of trainable parameters in the BERT
# Requires: keras-bert==0.81.0 keras-transformer==0.33.0
USE_ADAPTER = False
#USE_ADAPTER = True   

# Output pooling type extract/max/kmax
POOL_TYPE = "kmax"
POOL_K = 4  # k-max pooling parameter

# Specification of SCOPe classes used in classification
# See: https://scop.berkeley.edu/ver=2.07
TARGETS = "abcd"
#TARGETS = "abcdefg"

# Experimental data 
# Download from: http://bergerlab-downloads.csail.mit.edu/bepler-protein-sequence-embeddings-from-structure-iclr2019/scope.tar.gz
# See also: https://github.com/tbepler/protein-sequence-embedding-iclr2019#data-sets
TRAIN_DATA = "./SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.06.train.train.fa"
DEV_DATA = "./SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.06.train.dev.fa"
TEST_DATA = "./SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.06.test.fa"
TEST_DATA_NEW = "./SCOPe/astral-scopedom-seqres-gd-sel-gs-bib-95-2.07-new.fa"

# Maximum number of SentencePieces per example
MAX_LEN = 128
MAX_LEN1 = (MAX_LEN-2)  # Reserve two slots for [CLS] and [SEP] tokens

# Training parameters
BATCH_SIZE = 32   # Limited by the size of GPU
LR_START = 1e-3   # Initial learning rate, uses a simple Sqrt decay
EPOCHS = 3       # Number of training epochs

# SentencePieces allow sampling of different tokenizations
# Use PREDICT_AVG=1 to disable averaging and use the non-sampled tokenization only
PREDICT_AVG = 5

# Save the resulting model into this file
SAVE_MODEL_FN = "bert-sample-usage.hdf5"

### Custom class for K-max pooling

In [5]:
class KMaxPooling(keras.layers.Layer):
    """
    K-max pooling layer that extracts the k-highest activations from a sequence (2nd dimension).
    TensorFlow backend.
    """
    def __init__(self, k=1, axis=1, **kwargs):
        super(KMaxPooling, self).__init__(**kwargs)
        self.supports_masking = True
        self.input_spec = layers.InputSpec(ndim=3)
        self.k = k

        assert axis in [1,2],  'expected dimensions (samples, filters, convolved_values),\
                   cannot fold along samples dimension or axis not in list [1,2]'
        self.axis = axis

        # need to switch the axis with the last elemnet
        # to perform transpose for tok k elements since top_k works in last axis
        self.transpose_perm = [0,1,2] #default
        self.transpose_perm[self.axis] = 2
        self.transpose_perm[2] = self.axis
    
    def get_config(self):
        return {"k": self.k, "axis": self.axis}
   
    def build(self, input_shape):
        super(KMaxPooling, self).build(input_shape)
        self._my_output_shape = [i for i in input_shape]
        self._my_output_shape[0] = -1
        self._my_output_shape[self.axis] = self.k
        self._my_output_shape = tf.convert_to_tensor(self._my_output_shape)

    def call(self, x):
        # swap sequence dimension to get top k elements along axis=1
        transposed_for_topk = tf.transpose(x, perm=self.transpose_perm)

        # extract top_k, returns two tensors [values, indices]
        top_k_vals, top_k_indices = tf.math.top_k(transposed_for_topk,
                                                  k=self.k, sorted=True,
                                                  name=None)
        # maintain the order of values as in the paper
        # sort indices
        sorted_top_k_ind = tf.sort(top_k_indices)
        flatten_seq = tf.reshape(transposed_for_topk, (-1,))
        shape_seq = tf.shape(transposed_for_topk)
        len_seq = tf.shape(flatten_seq)[0]
        indices_seq = tf.range(len_seq)
        indices_seq = tf.reshape(indices_seq, shape_seq)
        indices_gather = tf.gather(indices_seq, 0, axis=-1)
        indices_sum = tf.expand_dims(indices_gather, axis=-1)
        sorted_top_k_ind += indices_sum
        k_max_out = tf.gather(flatten_seq, sorted_top_k_ind)
        # return back to normal dimension but now sequence dimension has only k elements
        # performing another transpose will get the tensor back to its original shape
        # but will have k as its axis_1 size
        transposed_back = tf.transpose(k_max_out, perm=self.transpose_perm)

        
        return tf.reshape(transposed_back, self._my_output_shape)



### Pre-trained BERT loading

Loads the pre-trained BERT model into Keras model.

See also: https://github.com/CyberZHG/keras-bert

In [6]:
config_path = os.path.join(CHECKPOINT_PATH, BERT_CONFIG)
model_path = os.path.join(CHECKPOINT_PATH, CHECKPOINT_NAME)
trainable = (['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(LAYER_NUM)] +
    ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(LAYER_NUM)] +
    ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(LAYER_NUM)] +
    ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(LAYER_NUM)] +
    ['Encoder-{}-MultiHeadSelfAttention'.format(i) for i in FINE_TUNE])


bert_model = load_trained_model_from_checkpoint(
    config_path,
    model_path,
    seq_len=MAX_LEN,
    training=False,
    trainable=trainable,
    **({} if USE_ADAPTER else {"use_adapter": True})
)

### SentencePiece model initialization

Loads the SentencePiece tokenizer from file and defines some helper functions.

See also: https://github.com/google/sentencepiece/blob/master/python/README.md

In [7]:
spm = sentencepiece.SentencePieceProcessor(SPM_MODEL)

def encode(seq, enable_sampling=True):
    if enable_sampling:
        ret = spm.encode(seq.upper(), out_type=str, enable_sampling=True, alpha=0.2, nbest_size=-1)
    else:
        ret = spm.encode(seq.upper(), out_type=str)
    ret = [i.lstrip("▁") for i in ret]
    ret = [i for i in ret if i]
    return ret

UNK = spm.PieceToId("[UNK]")

def PieceToId(piece):
    ret = spm.PieceToId(piece)
    if ret == 0:
        return UNK
    else:
        return ret

### Batch vectorization

Vectorizes the input batch, which is a list of (sequence, target_class) pairs.

Returns the tuple ([tokens, segments], targets) where:
 - tokens is an array the shape (batch_size, MAX_LEN) encoding the SentencePieces
 - segments is and array of zeros with the same shape
 - targets are vectorized softmax target values

In [8]:
def vectorize_batch(batch, enable_sampling=True):

    tokens = []
    segments = []
    targets = []
    for seqA, y in batch:
        if y not in TARGETS:
            continue
            
        tokens1 = []
        tokens1.append(PieceToId("[CLS]"))
        encoded = encode(seqA, enable_sampling)
        if len(encoded) > MAX_LEN1:
            if random.random() < 0.5:
                encoded = encoded[:MAX_LEN1]
            else:
                encoded = encoded[-MAX_LEN1:]
        tokens1.extend([PieceToId(i) for i in encoded])
        tokens1.append(PieceToId("[SEP]"))
         
        segments1 = [0]*len(tokens1)

        assert len(tokens1) == len(segments1)
        while len(tokens1) < MAX_LEN:
            tokens1.append(0)
            segments1.append(0)

        Y = [0]*len(TARGETS)
        Y[TARGETS.index(y)] = 1
            
        tokens.append(tokens1)
        segments.append(segments1)
        targets.append(Y)
    
    tokens = np.array(tokens, dtype=np.int32)
    segments = np.array(segments, dtype=np.int32)
    targets = np.array(targets)
    return [tokens, segments], targets

### Model definition

In [9]:
input_tokens = Input(shape=(MAX_LEN,), dtype='int32')
input_segments = Input(shape=(MAX_LEN,), dtype='int32')

bert_out = bert_model([input_tokens, input_segments])

bert_out = Dropout(0.25, noise_shape=(None, 128, 1))(bert_out)

dense_out = TimeDistributed(Dense(256, activation="relu"))(bert_out)

if POOL_TYPE == "extract":
    pool_out = Extract(index=0)(dense_out)
elif POOL_TYPE == "max":
    pool_out = MaskedGlobalMaxPool1D()(dense_out)
elif POOL_TYPE == "kmax":    
    pool_out = KMaxPooling(k=POOL_K)(dense_out)
    pool_out = Flatten()(pool_out)

out = Dense(len(TARGETS))(pool_out)
out = keras.layers.Activation("softmax")(out)

In [10]:
model = Model(inputs=[input_tokens, input_segments],
              outputs=[out])
model.compile(optimizer="adam", loss=["categorical_crossentropy"], metrics=["accuracy"])

In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, 128, 768)     93116328    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 128, 768)     0           model_1[0][0]              

### Dataset loading functions

In [12]:
def iter_training_sample(fn):
    ret = []
    for seq_record in SeqIO.parse(fn, "fasta"):
        struct = seq_record.description.split()[1]
        seq = str(seq_record.seq)
        ret.append((seq.upper(), struct[0]))
    random.seed(fn)
    random.shuffle(ret)
    return ret

def train_gen(data, batch_size, shuffle=True, enable_sampling=True):
    while True:
        data_it = data[:]
        if shuffle:
            random.shuffle(data_it)
        for idx0 in range(0, len(data_it), batch_size):
            batch = data_it[idx0:idx0+batch_size]
            yield vectorize_batch(batch, enable_sampling=enable_sampling)

In [13]:
train_data = iter_training_sample(TRAIN_DATA)
train_iter = train_gen(train_data, BATCH_SIZE)
TRAIN_STEPS = len(train_data) // BATCH_SIZE

dev_data = iter_training_sample(DEV_DATA)
dev_iter = train_gen(dev_data, BATCH_SIZE, enable_sampling=False)
DEV_STEPS = len(dev_data) // BATCH_SIZE

### Model training

In [14]:
callbacks = [LearningRateScheduler(lambda n: LR_START*1/((n+1)**0.5), verbose=1)]

model.fit(train_iter, steps_per_epoch=TRAIN_STEPS, epochs=EPOCHS,
          validation_data=dev_iter,
          validation_steps=DEV_STEPS,
          callbacks=callbacks)

Epoch 1/3

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
630/630 [==============================] - 444s 671ms/step - loss: 0.6662 - accuracy: 0.7799 - val_loss: 0.1416 - val_accuracy: 0.9489
Epoch 2/3

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0007071067811865475.
630/630 [==============================] - 418s 664ms/step - loss: 0.1421 - accuracy: 0.9496 - val_loss: 0.1200 - val_accuracy: 0.9664
Epoch 3/3

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0005773502691896258.
630/630 [==============================] - 418s 664ms/step - loss: 0.1108 - accuracy: 0.9616 - val_loss: 0.1037 - val_accuracy: 0.9674


### Predictions on test data

In [15]:
test_data = iter_training_sample(TEST_DATA)

pred_Y = 0
for i in range(PREDICT_AVG):
    test_X, test_Y = next(train_gen(test_data, len(test_data), shuffle=False, enable_sampling=(i!=0)))
    pred_Y = pred_Y + model.predict(test_X, batch_size=128, verbose=True)

test_Y1 = test_Y.argmax(axis=1)
pred_Y1 = pred_Y.argmax(axis=1)
print("Model accuracy", accuracy_score(test_Y1, pred_Y1))

40/40 [==============================] - 49s 1s/step
Model accuracy 0.965639112507363


In [16]:
print("Confusion matrix")
pd.DataFrame(confusion_matrix(test_Y1, pred_Y1),
             columns=list(TARGETS),
             index=list(TARGETS)).round(2)


Confusion matrix


,a,b,c,d
a,862,0,0,26
b,1,1323,4,55
c,5,3,1548,22
d,14,29,16,1185


In [17]:
test_data = iter_training_sample(TEST_DATA_NEW)

pred_Y = 0
for i in range(PREDICT_AVG):
    test_X, test_Y = next(train_gen(test_data, len(test_data), shuffle=False, enable_sampling=(i!=0)))
    pred_Y = pred_Y + model.predict(test_X, batch_size=128, verbose=True)

test_Y1 = test_Y.argmax(axis=1)
pred_Y1 = pred_Y.argmax(axis=1)
print("Model accuracy", accuracy_score(test_Y1, pred_Y1))

5/5 [==============================] - 6s 1s/step
Model accuracy 0.9462540716612378


In [18]:
model.save(SAVE_MODEL_FN)

## Copy out the resulting model into Google Drive

Clink on the displayed link and authorize the app to access your google drive.

In [20]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


Modify the target directory according to your needs.

In [23]:
!cp $SAVE_MODEL_FN gdrive/MyDrive/Models/